# Caso 5: localización de depósitos con múltiples objetivos

---
## Maestría en Inteligencia Analítica para la Toma de Decisiones

* **Profesor**: 
    * *Andrés Medaglia*
* **Asistentes**:
    * *Ariel Rojas*
    * *Carlos Arroyo*

## Instrucciones generales

El primer paso antes de resolver este laboratorio es leer y entender el **enunciado del caso**.

Este laboratorio tiene las siguientes secciones: 
* **Formulación**: en este caso particular, definimos dos funciones objetivo $z_1$ y $z_2$
* **Importación de librerías**
* **Creación de parámetros**
* **Modelado**: en esta práctica, haremos tres (3) implementaciones del mismo problema:
    * **Minimización de costos**
    * **Maximización de la satisfacción**
    * **Maximización de la satisfacción con restricción de costos**
* **Reporte de Resultados**

Este tipo de actividades se evaluará sobre un total de 100 puntos. Las celdas calificables se distinguen por tener la instrucción `# your code here`. Antes de estas celdas encontrarás instrucciones y consejos para resolver las preguntas, también el puntaje que le corresponde.

¡Éxitos!

## Formulación
---

Te presentamos la formulación del caso de la semana de forma resumida. Te recomendamos revisar la formulación una vez hayas leído el enunciado del caso. Es bueno que te familiarices con los elementos de la formulación antes de iniciar la implementación.

### Conjuntos y Parámetros
>#### **Conjuntos**
>* $I:$ Depósitos
>* $J:$ Centros de acopio consolidados (CACs)

>#### **Parámetros**
>* $k_i:$ Capacidad del depósito $i \in I$ (miles de toneladas por año)
>* $f_i:$ Costo de operación del depósito $i \in I$ (millones de pesos por año)
>* $d_j:$ Producción proyectada del CAC $j\in J$ (miles de tonaledas por año)
>* $q:$ Costo anualizado (por cada mil toneladas por kilómetro) de transportar café
>* $r:$ Distancia máxima (en kilómetros) entre el CAC y el depósito asignado para estar "bien" atendido
>* $h_{ij}:$ Distancia (en kilómetros) entre el CAC $j \in J$ y el depósito $i \in I$
>* $c_{ij}:$ Costo anualizado de atender el CAC $j\in J$ con el depósito $i\in I$ (Se calcula como: $c_{ij} = q\cdot d_j \cdot h_{ij}$)

### Variables de Decisión
>* $x_{ij}=\begin{cases}1, & \text{si el CAC } j \in J \text{ es atendido por el depósito } i \in I \\0, & \text{de lo contrario} \end{cases}$
>* $y_{i}=\begin{cases} 1, & \text{si se decide operar el depósito } i \in I  \\ 0, & \text{de lo contrario} \end{cases}$
    
### Restricciones
>1. Cada CAC debe ser atendido por un único depósito 
>>`# Para desarrollo del estudiante`
>2. No se debe superar la capacidad de los depósitos y sólo se puede atender CACs desde un depósito si se decide operar el mismo.
>>$\sum_{j \in J}d_{j}x_{ij} \leq k_{i}y_{i}, \; \forall i \in I$

> **Naturaleza de variables**
>>$x_{ij} \in \{0,1\} , \;\forall i \in I, j \in J$
>>
>>$y_{i} \in \{0,1\} , \;\forall i \in I$

### Función Objetivo
>* Minimizar los costos totales de operación y transporte
>>`# Para desarrollo del estudiante`
>* Maximizar satisfacción de los CACs
>>$\max z_2 = \sum_{j \in J} d_{j} \sum_{\{i \in I | h_{ij} \leq r\}} x_{ij}$

## Importación de librerías
---
En esta práctica usaremos:
* El paquete `pandas` es muy útil para el análisis de datos en general. Le asignamos el alias de `pd`.
* El paquete `pulp` permite crear modelos de optimización, crear variables, añadir restricciones y muchos más. Le asignamos el alias de `lp`.
* La función `distance` del módulo `geopy.distance` nos permite hallar fácilmente la distancia geodéisca en kilómetros entre dos pares de coordenadas de longitud y latitud.


In [ ]:
import pandas as pd
import pulp as lp 
from geopy.distance import distance

## Creación de Parámetros
---

### Lectura del archivo de soporte

Los datos que necesitamos para esta práctica se encuentran disponibles en el archivo `Soporte Caso 5.xlsx`.
En este archivo encontraremos los mismo datos del enunciado.
Importamos las hojas `CACs` y `Depositos` del archivo `Soporte Caso 5.xlsx`.
Estas hojas son importadas como objetos `DataFrame` de `pandas`.

In [ ]:
cacs = pd.read_excel('Soporte Caso 5.xlsx', sheet_name='CACs')
depositos = pd.read_excel('Soporte Caso 5.xlsx', sheet_name='Depositos')

### Procesamiento de archivos de soporte

En este paso, se crean los **Conjuntos** y **Parámetros**.
Es necesario dejar todo expresado en términos de listas y diccionarios para facilitar la implementación del modelo en PuLP. Adicionalmente, debemos procesar las coordenadas de longitud y latitud para obtener las distancias entre CACs y Depósitos.

In [ ]:
I = depositos.Municipio.to_list()
J = cacs.Municipio.to_list()

capacidad = {row["Municipio"]: row["Capacidad"] for _, row in depositos.iterrows()}
costo_fijo = {row["Municipio"]: row["CostoFijo"] for _, row in depositos.iterrows()}
depositos_lat_lon = {
    row["Municipio"]: (row["Latitud"], row["Longitud"])
    for _, row in depositos.iterrows()
}

produccion = {row["Municipio"]: row["Produccion"] for _, row in cacs.iterrows()}
cacs_lat_lon = {
    row["Municipio"]: (row["Latitud"], row["Longitud"]) for _, row in cacs.iterrows()
}

q = 90  # Pesos anualizados por cada mil toneladas de café por kilómetro
r = 125  # Kilómetros

distancia = {
    (i, j): distance(depositos_lat_lon[i], cacs_lat_lon[j]).kilometers
    for i in I
    for j in J
}

**Pregunta 1 (10 puntos)**

* Crea el parámetro de costo de transporte $c_{ij}$ en un diccionario llamado `costo_transporte`
* Las **llaves** de este diccionario deben ser los pares $(i,j)$, es decir, (depósitos, CACs)
* Los **valores** de este diccionario deben ser los costos de transporte definidos en la formulación

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Celda de Prueba (0 puntos)**

Es una buena práctica imprimir algunos objetos que contienen los parámetros en la consola luego de crearlos. De esta forma puedes corregir errores y familiarizarte con las estructuras de datos que se van a utilizar. Puedes hacer estas pruebas en la celda a continuación.

* **Esta celda no es calificable**

In [ ]:
# your code here


## Modelado - Minimización de costos ($z_1$)
---

### Declaración del modelo

In [ ]:
problema = lp.LpProblem(sense=lp.LpMinimize)

### Variables de Decisión

>* $x_{ij}=\begin{cases}1, & \text{si el CAC } j \in J \text{ es atendido por el depósito } i \in I \\0, & \text{de lo contrario} \end{cases}$
>* $y_{i}=\begin{cases} 1, & \text{si se decide operar el depósito } i \in I  \\ 0, & \text{de lo contrario} \end{cases}$

In [ ]:
x = lp.LpVariable.dicts("atender", [(i, j) for i in I for j in J], lowBound = 0, cat=lp.LpBinary)
y = lp.LpVariable.dicts("operar", I, lowBound = 0, cat=lp.LpBinary)

### Función Objetivo
Minimizar los costos totales de operación y transporte
>`# Para desarrollo del estudiante`

**Pregunta 2 (10 puntos)**
* Crea la función objetivo y agrégala al modelo `problema`

> **Ejemplo**:
>> $ \sum_{i \in I}c_i x_i$
es equivalente a `lp.lpSum(c[i]*x[i] for i in I)`

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Restricciones

____
**Ejemplo**
> La siguiente restricción: $\sum_{i \in I} a_{ij} x_{ij} \geq 1, \; \forall j \in J$ es equivalente a:
>    * `for j in J:`
>        * `model += lp.lpSum(a[i,j]*x[i,j] for i in I) >= 1, 'R1_'+str(j)`
    
**Advertencia**: En `pulp` no es recomendable sobreescribir restricciones, entonces, si ya creaste una restricción y quieres crearla de nuevo para corregir algo, asegúrate de volver a crear el modelo `problema` desde el principio. (Nosotros haremos esto antes de calificar, no te preocupes)

**Pregunta 3 (10 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R1_'+str(<indice_del_para_todo>)` y añádela al modelo:
>1. Cada CAC debe ser atendido por un único depósito
>>`# Para desarrollo del estudiante`

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 4 (5 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R2_'+str(<indice_del_para_todo>)` y añádela al modelo:
>2. No se debe superar la capacidad de los depósitos y sólo se puede atender CACs desde un depósito si se decide operar el mismo.
>>$\sum_{j \in J}d_{j}x_{ij} \leq k_{i}y_{i}, \; \forall i \in I$

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Invocar el optimizador

In [ ]:
print(lp.LpStatus[problema.solve()])

## Reporte de resultados - Minimización de costos ($z_1$)
---

**Función objetivo $z_1$**

In [ ]:
z1 = lp.value(problema.objective)
min_costo = z1
print(f"Costo Total: ${z1: .2f}")
print(f"Costo Total Relativo al Mínimo Costo: {z1/min_costo*100: .2f}%")

**Función objetivo $z_2$**

**Pregunta 5 (5 puntos)**

* Guarda en una variable `z2` el valor de la satisfacción total dado por la expresión:
> $z_2 = \sum_{j \in J} d_{j} \sum_{\{i \in I | h_{ij} \leq r\}} x_{ij}$

**Recuerda que** en PuLP puedes usar la función `lp.value(<expresion>)` para evaluar una expresión, reemplazando los valores de las variables por aquellos de la solución óptima. Esta función sólo debe ser llamada luego de usar `<modelo>.solve()` y haber obtenido una solución óptima.

In [ ]:
# your code here


In [ ]:
print(f"Satisfacción Total:{z2: .2f}")
print(
    f"Satisfacción Total Relativa al Total de Producción: {z2 / sum(produccion.values()) * 100 :.2f}%"
)

In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Depósitos en operación**

In [ ]:
print("Se decidió operar", sum(y[i].value() for i in I), "depósitos")

**Asignación de CACs a Depósitos**

In [ ]:
matrix = []
for j in J:
    row = []
    for i in I:
        if y[i].value() == 1:
            if x[i,j].value() == 1:
                row.append('X')
            elif x[i,j].value() == 0:
                row.append('-')
            else:
                row.append('Error')
    matrix.append(row)
    
df = pd.DataFrame(matrix, index=J, columns=[i for i in I if y[i].value() == 1])
df.head(10)

### Visualizaciones
---

**Mapa de la asignación**

In [ ]:
# Para los mapas
import folium
# Para los marcadores de los mapas
from folium.plugins import BeautifyIcon

m = folium.Map(location=[6.2, -74.5], tiles="OpenStreetMap", zoom_start=6)

for j, lat_lon in cacs_lat_lon.items():
    folium.Marker(
        location=lat_lon,
        tooltip=j,
        icon=BeautifyIcon(
            icon="circle",
            inner_icon_style="color:blue;font-size:7px;opacity:0.9;position: relative;top:-0.5px;",
            background_color="transparent",
            border_color="transparent",
        ),
    ).add_to(m)
for i, lat_lon in depositos_lat_lon.items():
    if y[i].value() > 0:
        folium.Marker(
            location=lat_lon,
            tooltip=i,
            icon=BeautifyIcon(
                icon="caret-up",
                inner_icon_style="color:red;font-size:20px;opacity:0.9;position: relative;top:-4.5px;",
                background_color="transparent",
                border_color="transparent",
            ),
        ).add_to(m)

red = [(i, j) for i in I for j in J if x[i, j].value() > 0]
for i, j in red:
    folium.PolyLine(
        [depositos_lat_lon[i], cacs_lat_lon[j]], color="black", weight=1, opacity=1
    ).add_to(m)

m

## Modelado - Maximización de satisfacción ($z_2$)
---
A continuación queremos explorar el cambio en las funciones objetivo $z_1$ y $z_2$ cuando se prioriza $z_2$. Las restricciones y variables del problema permanecen igual, pero la solución cambiará.

### Declaración del modelo

In [ ]:
problema = lp.LpProblem(sense=lp.LpMaximize)

### Variables de Decisión

>* $x_{ij}=\begin{cases}1, & \text{si el CAC } j \in J \text{ es atendido por el depósito } i \in I \\0, & \text{de lo contrario} \end{cases}$
>* $y_{i}=\begin{cases} 1, & \text{si se decide operar el depósito } i \in I  \\ 0, & \text{de lo contrario} \end{cases}$

In [ ]:
x = lp.LpVariable.dicts('atender', [(i,j) for i in I for j in J], cat=lp.LpBinary)
y = lp.LpVariable.dicts('operar', I, cat=lp.LpBinary)

### Función Objetivo
Maximizar satisfacción de los CACs
>$\max z_2 = \sum_{j \in J} d_{j} \sum_{\{i \in I | h_{ij} \leq r\}} x_{ij}.$

Esta expresión es equivalente a:
>$\max z_2 = \sum_{j \in J} \sum_{\{i \in I | h_{ij} \leq r\}}d_{j} x_{ij}.$

**Pregunta 6 (5 puntos)**
* Crea la función objetivo y agrégala al modelo `problema`

> **Ejemplo**:
>> $ \sum_{i \in I}c_i x_i$
es equivalente a `lp.lpSum(c[i]*x[i] for i in I)`

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Restricciones
____

**Pregunta 7 (10 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R1_'+str(<indice_del_para_todo>)` y añádela al modelo:
>1. Cada CAC debe ser atendido por un único depósito
>>`# Para desarrollo del estudiante`

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 8 (5 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R2_'+str(<indice_del_para_todo>)` y añádela al modelo:
>2. No se debe superar la capacidad de los depósitos y sólo se puede atender CACs desde un depósito si se decide operar el mismo.
>>$\sum_{j \in J}d_{j}x_{ij} \leq k_{i}y_{i}, \; \forall i \in I$

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Invocar el optimizador

In [ ]:
print(lp.LpStatus[problema.solve()])

## Reporte de resultados - Maximización de satisfacción ($z_2$)
---

**Función objetivo $z_1$**

**Pregunta 9 (5 puntos)**

* Guarda en una variable `z1` el valor del costo total de operación y transporte:
>`# Para desarrollo del estudiante`

**Recuerda que** en PuLP puedes usar la función `lp.value(<expresion>)` para evaluar una expresión, reemplazando los valores de las variables por aquellos de la solución óptima. Esta función sólo debe ser llamada luego de usar `<modelo>.solve()` y haber obtenido una solución óptima.

In [ ]:
# your code here


In [ ]:
print(f"Costo Total: ${z1: .2f}")
print(f"Costo Total Relativo al Mínimo Costo: {z1 / min_costo * 100: .2f}%")

In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Función objetivo $z_2$**

In [ ]:
z2 = lp.value(problema.objective)
print(f"Satisfacción Total: {z2: .2f}")
print(
    f"Satisfacción Total Relativa al Total de Producción: {z2 / sum(produccion.values()) * 100: .2f}%"
)

**Depósitos en operación**

In [ ]:
print("Se decidió operar", sum([y[i].value() for i in I]), "depósitos")

**Asignación de CACs a Depósitos**

In [ ]:
matrix = []
for j in J:
    row = []
    for i in I:
        if y[i].value() == 1:
            if x[i, j].value() == 1:
                row.append("X")
            elif x[i, j].value() == 0:
                row.append("-")
            else:
                row.append("Error")
    matrix.append(row)

df = pd.DataFrame(matrix, index=J, columns=[i for i in I if y[i].value() == 1])
df.head(10)

### Visualizaciones
---

**Mapa de la asignación**

In [ ]:
m = folium.Map(location=[6.2, -74.5], tiles="OpenStreetMap", zoom_start=6)

for j, lat_lon in cacs_lat_lon.items():
    folium.Marker(
        location=lat_lon,
        tooltip=j,
        icon=BeautifyIcon(
            icon="circle",
            inner_icon_style="color:blue;font-size:7px;opacity:0.9;position: relative;top:-0.5px;",
            background_color="transparent",
            border_color="transparent",
        ),
    ).add_to(m)
for i, lat_lon in depositos_lat_lon.items():
    if y[i].value() > 0:
        folium.Marker(
            location=lat_lon,
            tooltip=i,
            icon=BeautifyIcon(
                icon="caret-up",
                inner_icon_style="color:red;font-size:20px;opacity:0.9;position: relative;top:-4.5px;",
                background_color="transparent",
                border_color="transparent",
            ),
        ).add_to(m)

red = [(i, j) for i in I for j in J if x[i, j].value() > 0]
for i, j in red:
    folium.PolyLine(
        [depositos_lat_lon[i], cacs_lat_lon[j]], color="black", weight=1, opacity=1
    ).add_to(m)

m

## Modelado - Maximización de satisfacción ($z_2$) con restricción de costos ($z_1$)
---
Por último, queremos encontrar un solución intermedia entre la que minimiza los costos y la que maximiza la satisfacción. Hay varias maneras de hacer esto. La que vamos a implementar es colocar una restricción sobre los costos $z_1$ que esté entre los valores obtenidos en los dos casos anteriores.

Recordemos que al minimizar los costos, se obtuvo un costo total de 4,318,336.74. Este costo será nuestro punto de referencia. No podemos obtener un costo menor a este. Por otro lado, al maximizar la satisfacción, obtuvimos un costo de 4,837,569.38. Así que en el peor de los casos, el costo es aproximadamente 12.02% mayor al primer caso. Entonces, para obtener una solución intermedia, debemos escoger un umbral entre estos dos valores para crear una restricción sobre los costos. Una posibilidad es restringir que el costo total $z_1$ sea a lo sumo 2% mayor que el mejor costo mientras se maximiza la satisfacción $z_2$. 

### Declaración del modelo

In [ ]:
problema = lp.LpProblem(sense=lp.LpMaximize)

### Variables de Decisión

>* $x_{ij}=\begin{cases}1, & \text{si el CAC } j \in J \text{ es atendido por el depósito } i \in I \\0, & \text{de lo contrario} \end{cases}$
>* $y_{i}=\begin{cases} 1, & \text{Si se decide operar el depósito } i \in I  \\ 0, & \text{de lo contrario} \end{cases}$

In [ ]:
x = lp.LpVariable.dicts("atender", [(i, j) for i in I for j in J], cat=lp.LpBinary)
y = lp.LpVariable.dicts("operar", I, cat=lp.LpBinary)

### Función Objetivo
Maximizar satisfacción de los CACs
>$\max z_2 = \sum_{j \in J} d_{j} \sum_{\{i \in I | h_{ij} \leq r\}} x_{ij}$

**Pregunta 10 (5 puntos)**
* Crea la función objetivo y agrégala al modelo `problema`

> **Ejemplo**:
>> $ \sum_{i \in I}c_i x_i$
es equivalente a `lp.lpSum(c[i]*x[i] for i in I)`

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Restricciones
____

**Pregunta 11 (10 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R1_'+str(<indice_del_para_todo>)` y añádela al modelo:
>1. Cada CAC debe ser atendido por un único depósito
>>`# Para desarrollo del estudiante`

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 12 (5 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R2_'+str(<indice_del_para_todo>)` y añádela al modelo:
>2. No se debe superar la capacidad de los depósitos y sólo se puede atender CACs desde un depósito si se decide operar el mismo.
>>$\sum_{j \in J}d_{j}x_{ij} \leq k_{i}y_{i}, \; \forall i \in I$

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Pregunta 13 (10 puntos)**

* Crea la siguiente restricción, asígnale el nombre `'R3'` y añádela al modelo:
>El costo total no debe superar en más de un 2% al mejor costo obtenido.
>>`# Para desarrollo del estudiante` 

**Nota:** Utiliza el valor `z1_` a continuación como el mejor costo obtenido. Inclúyelo en la restricción según sea conveniente.

In [ ]:
z1_ = 4318336.74

In [ ]:
# your code here


In [ ]:
# Esta celda esta reservada para uso del equipo docente

In [ ]:
# Esta celda esta reservada para uso del equipo docente

### Invocar el optimizador

In [ ]:
print(lp.LpStatus[problema.solve()])

## Reporte de resultados - Maximización de satisfacción ($z_2$) con restricción de costos ($z_1$)
---

**Función objetivo $z_1$**

**Pregunta 14 (5 puntos)**

* Guarda en una variable `z1` el valor del costo total de operación y transporte:
>`# Para desarrollo del estudiante`

**Recuerda que** en PuLP puedes usar la función `lp.value(<expresion>)` para evaluar una expresión, reemplazando los valores de las variables por aquellos de la solución óptima. Esta función sólo debe ser llamada luego de usar `<modelo>.solve()` y haber obtenido una solución óptima.

In [ ]:
# your code here


In [ ]:
print(f"Costo Total: ${z1: .2f}")
print(f"Costo Total Relativo al Mínimo Costo: {z1 / min_costo * 100: .2f}%")

In [ ]:
# Esta celda esta reservada para uso del equipo docente

**Función objetivo $z_2$**

In [ ]:
z2 = lp.value(problema.objective)
print(f"Satisfacción Total: {z2: .2f}")
print(f"Satisfacción Total Relativa al Total de Producción: {z2 / sum(produccion.values()) * 100: .2f}%")

**Depósitos en operación**

In [ ]:
print("Se decidió operar", sum(y[i].value() for i in I), "depósitos")

**Asignación de CACs a Depósitos**

In [ ]:
matrix = []
for j in J:
    row = []
    for i in I:
        if y[i].value() == 1:
            if x[i, j].value() == 1:
                row.append("X")
            elif x[i, j].value() == 0:
                row.append("-")
            else:
                row.append("Error")
    matrix.append(row)

df = pd.DataFrame(matrix, index=J, columns=[i for i in I if y[i].value() == 1])
df.head(10)

### Visualizaciones
---

**Mapa de la asignación**

In [ ]:
m = folium.Map(location=[6.2, -74.5], tiles="OpenStreetMap", zoom_start=6)

for j, lat_lon in cacs_lat_lon.items():
    folium.Marker(
        location=lat_lon,
        tooltip=j,
        icon=BeautifyIcon(
            icon="circle",
            inner_icon_style="color:blue;font-size:7px;opacity:0.9;position: relative;top:-0.5px;",
            background_color="transparent",
            border_color="transparent",
        ),
    ).add_to(m)

for i, lat_lon in depositos_lat_lon.items():
    if y[i].value() > 0:
        folium.Marker(
            location=lat_lon,
            tooltip=i,
            icon=BeautifyIcon(
                icon="caret-up",
                inner_icon_style="color:red;font-size:20px;opacity:0.9;position: relative;top:-4.5px;",
                background_color="transparent",
                border_color="transparent",
            ),
        ).add_to(m)

red = [(i, j) for i in I for j in J if x[i, j].value() > 0]
for i, j in red:
    folium.PolyLine(
        [depositos_lat_lon[i], cacs_lat_lon[j]], color="black", weight=1, opacity=1
    ).add_to(m)
m

### Reflexión
---
¿De qué forma podrías obtener soluciones intermedias adicionales? ¿Podrías presentarlas gráficamente como una frontera de Pareto? ¿Si tuvieras que recomendar alguna solución, con qué criterio la escogerías?

### Fin del laboratorio
---
¡Muchos éxitos!